In [9]:
import requests
import time
import os
from typing import List, Optional
from urllib.parse import urlparse, parse_qs

def get_my_replay_ids(
    api_key: str, 
    max_pages: Optional[int] = None
) -> List[str]:
    """
    Retrieve Replay IDs for a specific player from Ballchasing API using URL-based pagination.
    """
    base_url = "https://ballchasing.com/api/replays"
    replay_ids = []
    page = 0
    current_url = base_url
    
    # Initial parameters
    params = {
        "player-id": 'epic:42ae53b8b0d640328bae991be8edfcde',
        "playlist": 'ranked-doubles',
        "sort-by": "created",
        "sort-dir": "desc"
    }
    
    headers = {
        "Authorization": api_key
    }

    while True:
        try:
            print(f"\n=== Fetching Page {page + 1} ===")
            
            # For first page, use parameters. For subsequent pages, use the full URL
            if page == 0:
                response = requests.get(base_url, headers=headers, params=params)
            else:
                response = requests.get(current_url, headers=headers)
            
            response.raise_for_status()
            data = response.json()
            
            # Print total count on first page
            if page == 0:
                total_count = data.get('count', 0)
                print(f"Total replays available: {total_count}")
            
            # Extract replay IDs
            page_replay_ids = [replay['id'] for replay in data.get('list', [])]
            print(f"Replays found on this page: {len(page_replay_ids)}")
            
            if not page_replay_ids:
                print("No replays found on this page.")
                break
            
            replay_ids.extend(page_replay_ids)
            
            # Get the next URL from the response
            next_url = data.get('next')
            if not next_url:
                print("No next page URL found.")
                break
                
            current_url = next_url
            print(f"Next URL: {current_url}")
            
            page += 1
            if max_pages and page >= max_pages:
                print(f"\nReached maximum pages limit: {max_pages}")
                break
            
            print(f"Total replays retrieved so far: {len(replay_ids)}")
            time.sleep(1)  # Respect rate limits
            
        except requests.RequestException as e:
            print(f"Error retrieving replays on page {page}: {e}")
            if 'response' in locals():
                print(f"Response content: {response.text}")
            break
    
    return replay_ids

def main():
    API_KEY = os.getenv('API_KEY')
    
    print("=== Starting Replay ID Retrieval ===")
    replay_ids = get_my_replay_ids(API_KEY, max_pages=None)  # Set to None to get all pages
    
    print("\n=== Final Results ===")
    print(f"Total Replay IDs retrieved: {len(replay_ids)}")
    print(f"Unique Replay IDs: {len(set(replay_ids))}")
    
    if replay_ids:
        print("\nFirst 5 replay IDs:")
        for i, rid in enumerate(replay_ids[:5]):
            print(f"{i+1}. {rid}")
        
        print("\nLast 5 replay IDs:")
        for i, rid in enumerate(replay_ids[-5:]):
            print(f"{i+1}. {rid}")
    
    return replay_ids

if __name__ == "__main__":
    replay_ids = main()

=== Starting Replay ID Retrieval ===

=== Fetching Page 1 ===
Total replays available: 1386
Replays found on this page: 50
Next URL: https://ballchasing.com/api/replays?after=MS43MzI1ODcwNTYyMDJlKzEyXzMwYWMwMmRkLTg0ZWItNDhlMS1iMDVhLWUxNWQ5NzY0ODM4MQ%3D%3D&player-id=epic%3A42ae53b8b0d640328bae991be8edfcde&playlist=ranked-doubles&sort-by=created&sort-dir=desc
Total replays retrieved so far: 50

=== Fetching Page 2 ===
Replays found on this page: 50
Next URL: https://ballchasing.com/api/replays?after=MS43MzA3NTcxMDYzMTNlKzEyX2VjM2VjN2RiLTc4OWEtNGU1ZC1iODk1LWFjZmQzZjllOTBmOA%3D%3D&player-id=epic%3A42ae53b8b0d640328bae991be8edfcde&playlist=ranked-doubles&sort-by=created&sort-dir=desc
Total replays retrieved so far: 100

=== Fetching Page 3 ===
Replays found on this page: 50
Next URL: https://ballchasing.com/api/replays?after=MS43MjgyNjg1ODY3MzJlKzEyXzY2ZWQxZDkyLTBiNTQtNDcxMC1iZGE3LTVkYzg0N2ZiMGIxMQ%3D%3D&player-id=epic%3A42ae53b8b0d640328bae991be8edfcde&playlist=ranked-doubles&sort-by=creat

In [12]:
len(set(replay_ids))

1386

In [13]:
import csv
with open('replay_ids', 'w', newline='') as myfile:
     wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
     wr.writerow(replay_ids)